In [1]:
import march_madness_classes as mmc
import march_madness_games as mmg
import march_madness_models as mmm
import march_madness_train_and_tune as mmtt

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as LogReg

In [2]:
# read in the dataset
seeds = pd.read_csv("datasets/kaggle_data_2021/MNCAATourneySeeds.csv")
slots = pd.read_csv("datasets/kaggle_data_2021/MNCAATourneySlots.csv")
games = pd.read_csv("datasets/kaggle_data_2021/MNCAATourneyCompactResults.csv")

seeds_arr = mmg.filter_into_seasons(seeds)
slots_arr = mmg.filter_into_seasons(slots)
games_arr = mmg.filter_into_seasons(games)

In [3]:
# extract predictors, chosen from the variable selection notebook
markov          = pd.read_csv("datasets/our_data/stationary", index_col=0)
rpi             = pd.read_csv("datasets/our_data/rpi", index_col=0)
bad_losses      = pd.read_csv("datasets/our_data/bad_losses", index_col=0)

# seeds
seed_matrix_df  = pd.read_csv("datasets/our_data/team_summary_data/seeds_matrix", index_col=0)

In [4]:
# get data into correct format
predictor_names = ["min_index_id", "max_index_id", "markov","rpi","bad_losses"] 

# package the predictors into an array
predictor_dfs = [markov, rpi, bad_losses] 

In [5]:
train_test_arr = []

window   = 3
min_year = 2014
max_year = 2016

year_range = range(min_year, max_year)

# generate our train test split for each year
for year in year_range:
    # do the split for the current year
    train_test_tuple = mmtt.train_test_split(window, 
                                             year, 
                                             seeds_arr, 
                                             slots_arr, 
                                             games_arr, 
                                             predictor_names, 
                                             predictor_dfs)
    
    # add to our array
    train_test_arr.append(train_test_tuple)

Season        2011
Slot          R1W1
StrongSeed     W01
WeakSeed       W16
Name: 1648, dtype: object
Season        2011
Slot          R1W2
StrongSeed     W02
WeakSeed       W15
Name: 1649, dtype: object
Season        2011
Slot          R1W3
StrongSeed     W03
WeakSeed       W14
Name: 1650, dtype: object
Season        2011
Slot          R1W4
StrongSeed     W04
WeakSeed       W13
Name: 1651, dtype: object
Season        2011
Slot          R1W5
StrongSeed     W05
WeakSeed       W12
Name: 1652, dtype: object
Season        2011
Slot          R1W6
StrongSeed     W06
WeakSeed       W11
Name: 1653, dtype: object
Season        2011
Slot          R1W7
StrongSeed     W07
WeakSeed       W10
Name: 1654, dtype: object
Season        2011
Slot          R1W8
StrongSeed     W08
WeakSeed       W09
Name: 1655, dtype: object
Season        2011
Slot          R1X1
StrongSeed     X01
WeakSeed       X16
Name: 1656, dtype: object
Season        2011
Slot          R1X2
StrongSeed     X02
WeakSeed       X15
Name: 

Season        2014
Slot          R1W1
StrongSeed     W01
WeakSeed       W16
Name: 1849, dtype: object
Season        2014
Slot          R1W2
StrongSeed     W02
WeakSeed       W15
Name: 1850, dtype: object
Season        2014
Slot          R1W3
StrongSeed     W03
WeakSeed       W14
Name: 1851, dtype: object
Season        2014
Slot          R1W4
StrongSeed     W04
WeakSeed       W13
Name: 1852, dtype: object
Season        2014
Slot          R1W5
StrongSeed     W05
WeakSeed       W12
Name: 1853, dtype: object
Season        2014
Slot          R1W6
StrongSeed     W06
WeakSeed       W11
Name: 1854, dtype: object
Season        2014
Slot          R1W7
StrongSeed     W07
WeakSeed       W10
Name: 1855, dtype: object
Season        2014
Slot          R1W8
StrongSeed     W08
WeakSeed       W09
Name: 1856, dtype: object
Season        2014
Slot          R1X1
StrongSeed     X01
WeakSeed       X16
Name: 1857, dtype: object
Season        2014
Slot          R1X2
StrongSeed     X02
WeakSeed       X15
Name: 

Season        2014
Slot          R1W1
StrongSeed     W01
WeakSeed       W16
Name: 1849, dtype: object
Season        2014
Slot          R1W2
StrongSeed     W02
WeakSeed       W15
Name: 1850, dtype: object
Season        2014
Slot          R1W3
StrongSeed     W03
WeakSeed       W14
Name: 1851, dtype: object
Season        2014
Slot          R1W4
StrongSeed     W04
WeakSeed       W13
Name: 1852, dtype: object
Season        2014
Slot          R1W5
StrongSeed     W05
WeakSeed       W12
Name: 1853, dtype: object
Season        2014
Slot          R1W6
StrongSeed     W06
WeakSeed       W11
Name: 1854, dtype: object
Season        2014
Slot          R1W7
StrongSeed     W07
WeakSeed       W10
Name: 1855, dtype: object
Season        2014
Slot          R1W8
StrongSeed     W08
WeakSeed       W09
Name: 1856, dtype: object
Season        2014
Slot          R1X1
StrongSeed     X01
WeakSeed       X16
Name: 1857, dtype: object
Season        2014
Slot          R1X2
StrongSeed     X02
WeakSeed       X15
Name: 

In [6]:
# our cross validated value of c, from variable selection notebook
c = 1
variables = ["markov","rpi","bad_losses"] 

# models and scalers to be fit
models  = []
scalers = []

for year in year_range:
        # get train data
        train_x = train_test_arr[year - min_year][0][variables]
        train_y = train_test_arr[year - min_year][1].values[:, 0]

        # get cross validation set
        cross_x = train_test_arr[year - min_year][2][variables]
        cross_y = train_test_arr[year - min_year][3].values[:, 0]

        # scaling
        scaler = StandardScaler().fit(train_x)
        scaled_train_x = scaler.transform(train_x)
        scaled_cross_x = scaler.transform(cross_x)

        # init model
        model = LogReg(C=c)

        # fit model
        model.fit(scaled_train_x, train_y)
        
        # append to our lis of models
        models.append(model)
        scalers.append(scaler)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [7]:
reload(mmm)

NameError: name 'reload' is not defined

In [8]:
# setup simulator
simulators = []

i = 0
# iterate years
for year in year_range: 
    # get data needed
    seeds_year = seeds_arr[year-1985] 
    slots_year = slots_arr[year-1985] 
    games_year = games_arr[year-1985]
    
    # setup head to head model
    head_to_head_model_year = mmm.ModelPredictor(models[i], scalers[i], predictor_dfs, year, 
                                                 simulation=True,
                                                 higher_seed_bias=False,
                                                 seeds_df=seeds_year, 
                                                 higher_seed_bias_delta=.04)
    
    # setup simulator
    simulators.append(mmc.Simulator(seeds_year, slots_year, head_to_head_model_year))
    
    i = i + 1

In [10]:
# run simulations
i = 0
for year in year_range:
    # run simulation
    points = simulators[i].simulate_tournament(200) 
    
    # predict bracket based on the simulation
    bracket = simulators[i].predict_tournament()
    
    i = i + 1
    
    print(year)

Season        2014
Slot          R1W1
StrongSeed     W01
WeakSeed       W16
Name: 1849, dtype: object
Season        2014
Slot          R1W2
StrongSeed     W02
WeakSeed       W15
Name: 1850, dtype: object
Season        2014
Slot          R1W3
StrongSeed     W03
WeakSeed       W14
Name: 1851, dtype: object
Season        2014
Slot          R1W4
StrongSeed     W04
WeakSeed       W13
Name: 1852, dtype: object
Season        2014
Slot          R1W5
StrongSeed     W05
WeakSeed       W12
Name: 1853, dtype: object
Season        2014
Slot          R1W6
StrongSeed     W06
WeakSeed       W11
Name: 1854, dtype: object
Season        2014
Slot          R1W7
StrongSeed     W07
WeakSeed       W10
Name: 1855, dtype: object
Season        2014
Slot          R1W8
StrongSeed     W08
WeakSeed       W09
Name: 1856, dtype: object
Season        2014
Slot          R1X1
StrongSeed     X01
WeakSeed       X16
Name: 1857, dtype: object
Season        2014
Slot          R1X2
StrongSeed     X02
WeakSeed       X15
Name: 

Name: 1865, dtype: object
Season        2014
Slot          R1Y2
StrongSeed     Y02
WeakSeed       Y15
Name: 1866, dtype: object
Season        2014
Slot          R1Y3
StrongSeed     Y03
WeakSeed       Y14
Name: 1867, dtype: object
Season        2014
Slot          R1Y4
StrongSeed     Y04
WeakSeed       Y13
Name: 1868, dtype: object
Season        2014
Slot          R1Y5
StrongSeed     Y05
WeakSeed       Y12
Name: 1869, dtype: object
Season        2014
Slot          R1Y6
StrongSeed     Y06
WeakSeed       Y11
Name: 1870, dtype: object
Season        2014
Slot          R1Y7
StrongSeed     Y07
WeakSeed       Y10
Name: 1871, dtype: object
Season        2014
Slot          R1Y8
StrongSeed     Y08
WeakSeed       Y09
Name: 1872, dtype: object
Season        2014
Slot          R1Z1
StrongSeed     Z01
WeakSeed       Z16
Name: 1873, dtype: object
Season        2014
Slot          R1Z2
StrongSeed     Z02
WeakSeed       Z15
Name: 1874, dtype: object
Season        2014
Slot          R1Z3
StrongSeed     Z03

Name: 1871, dtype: object
Season        2014
Slot          R1Y8
StrongSeed     Y08
WeakSeed       Y09
Name: 1872, dtype: object
Season        2014
Slot          R1Z1
StrongSeed     Z01
WeakSeed       Z16
Name: 1873, dtype: object
Season        2014
Slot          R1Z2
StrongSeed     Z02
WeakSeed       Z15
Name: 1874, dtype: object
Season        2014
Slot          R1Z3
StrongSeed     Z03
WeakSeed       Z14
Name: 1875, dtype: object
Season        2014
Slot          R1Z4
StrongSeed     Z04
WeakSeed       Z13
Name: 1876, dtype: object
Season        2014
Slot          R1Z5
StrongSeed     Z05
WeakSeed       Z12
Name: 1877, dtype: object
Season        2014
Slot          R1Z6
StrongSeed     Z06
WeakSeed       Z11
Name: 1878, dtype: object
Season        2014
Slot          R1Z7
StrongSeed     Z07
WeakSeed       Z10
Name: 1879, dtype: object
Season        2014
Slot          R1Z8
StrongSeed     Z08
WeakSeed       Z09
Name: 1880, dtype: object
Season        2014
Slot          R1W1
StrongSeed     W01

Name: 1880, dtype: object
Season        2014
Slot          R1W1
StrongSeed     W01
WeakSeed       W16
Name: 1849, dtype: object
Season        2014
Slot          R1W2
StrongSeed     W02
WeakSeed       W15
Name: 1850, dtype: object
Season        2014
Slot          R1W3
StrongSeed     W03
WeakSeed       W14
Name: 1851, dtype: object
Season        2014
Slot          R1W4
StrongSeed     W04
WeakSeed       W13
Name: 1852, dtype: object
Season        2014
Slot          R1W5
StrongSeed     W05
WeakSeed       W12
Name: 1853, dtype: object
Season        2014
Slot          R1W6
StrongSeed     W06
WeakSeed       W11
Name: 1854, dtype: object
Season        2014
Slot          R1W7
StrongSeed     W07
WeakSeed       W10
Name: 1855, dtype: object
Season        2014
Slot          R1W8
StrongSeed     W08
WeakSeed       W09
Name: 1856, dtype: object
Season        2014
Slot          R1X1
StrongSeed     X01
WeakSeed       X16
Name: 1857, dtype: object
Season        2014
Slot          R1X2
StrongSeed     X02

Name: 1867, dtype: object
Season        2014
Slot          R1Y4
StrongSeed     Y04
WeakSeed       Y13
Name: 1868, dtype: object
Season        2014
Slot          R1Y5
StrongSeed     Y05
WeakSeed       Y12
Name: 1869, dtype: object
Season        2014
Slot          R1Y6
StrongSeed     Y06
WeakSeed       Y11
Name: 1870, dtype: object
Season        2014
Slot          R1Y7
StrongSeed     Y07
WeakSeed       Y10
Name: 1871, dtype: object
Season        2014
Slot          R1Y8
StrongSeed     Y08
WeakSeed       Y09
Name: 1872, dtype: object
Season        2014
Slot          R1Z1
StrongSeed     Z01
WeakSeed       Z16
Name: 1873, dtype: object
Season        2014
Slot          R1Z2
StrongSeed     Z02
WeakSeed       Z15
Name: 1874, dtype: object
Season        2014
Slot          R1Z3
StrongSeed     Z03
WeakSeed       Z14
Name: 1875, dtype: object
Season        2014
Slot          R1Z4
StrongSeed     Z04
WeakSeed       Z13
Name: 1876, dtype: object
Season        2014
Slot          R1Z5
StrongSeed     Z05

...

Name: 1938, dtype: object
Season        2015
Slot          R1Y8
StrongSeed     Y08
WeakSeed       Y09
Name: 1939, dtype: object
Season        2015
Slot          R1Z1
StrongSeed     Z01
WeakSeed       Z16
Name: 1940, dtype: object
Season        2015
Slot          R1Z2
StrongSeed     Z02
WeakSeed       Z15
Name: 1941, dtype: object
Season        2015
Slot          R1Z3
StrongSeed     Z03
WeakSeed       Z14
Name: 1942, dtype: object
Season        2015
Slot          R1Z4
StrongSeed     Z04
WeakSeed       Z13
Name: 1943, dtype: object
Season        2015
Slot          R1Z5
StrongSeed     Z05
WeakSeed       Z12
Name: 1944, dtype: object
Season        2015
Slot          R1Z6
StrongSeed     Z06
WeakSeed       Z11
Name: 1945, dtype: object
Season        2015
Slot          R1Z7
StrongSeed     Z07
WeakSeed       Z10
Name: 1946, dtype: object
Season        2015
Slot          R1Z8
StrongSeed     Z08
WeakSeed       Z09
Name: 1947, dtype: object
Season        2015
Slot          R1W1
StrongSeed     W01

Name: 1925, dtype: object
Season        2015
Slot          R1X3
StrongSeed     X03
WeakSeed       X14
Name: 1926, dtype: object
Season        2015
Slot          R1X4
StrongSeed     X04
WeakSeed       X13
Name: 1927, dtype: object
Season        2015
Slot          R1X5
StrongSeed     X05
WeakSeed       X12
Name: 1928, dtype: object
Season        2015
Slot          R1X6
StrongSeed     X06
WeakSeed       X11
Name: 1929, dtype: object
Season        2015
Slot          R1X7
StrongSeed     X07
WeakSeed       X10
Name: 1930, dtype: object
Season        2015
Slot          R1X8
StrongSeed     X08
WeakSeed       X09
Name: 1931, dtype: object
Season        2015
Slot          R1Y1
StrongSeed     Y01
WeakSeed       Y16
Name: 1932, dtype: object
Season        2015
Slot          R1Y2
StrongSeed     Y02
WeakSeed       Y15
Name: 1933, dtype: object
Season        2015
Slot          R1Y3
StrongSeed     Y03
WeakSeed       Y14
Name: 1934, dtype: object
Season        2015
Slot          R1Y4
StrongSeed     Y04

Name: 1946, dtype: object
Season        2015
Slot          R1Z8
StrongSeed     Z08
WeakSeed       Z09
Name: 1947, dtype: object
Season        2015
Slot          R1W1
StrongSeed     W01
WeakSeed       W16
Name: 1916, dtype: object
Season        2015
Slot          R1W2
StrongSeed     W02
WeakSeed       W15
Name: 1917, dtype: object
Season        2015
Slot          R1W3
StrongSeed     W03
WeakSeed       W14
Name: 1918, dtype: object
Season        2015
Slot          R1W4
StrongSeed     W04
WeakSeed       W13
Name: 1919, dtype: object
Season        2015
Slot          R1W5
StrongSeed     W05
WeakSeed       W12
Name: 1920, dtype: object
Season        2015
Slot          R1W6
StrongSeed     W06
WeakSeed       W11
Name: 1921, dtype: object
Season        2015
Slot          R1W7
StrongSeed     W07
WeakSeed       W10
Name: 1922, dtype: object
Season        2015
Slot          R1W8
StrongSeed     W08
WeakSeed       W09
Name: 1923, dtype: object
Season        2015
Slot          R1X1
StrongSeed     X01

Name: 1933, dtype: object
Season        2015
Slot          R1Y3
StrongSeed     Y03
WeakSeed       Y14
Name: 1934, dtype: object
Season        2015
Slot          R1Y4
StrongSeed     Y04
WeakSeed       Y13
Name: 1935, dtype: object
Season        2015
Slot          R1Y5
StrongSeed     Y05
WeakSeed       Y12
Name: 1936, dtype: object
Season        2015
Slot          R1Y6
StrongSeed     Y06
WeakSeed       Y11
Name: 1937, dtype: object
Season        2015
Slot          R1Y7
StrongSeed     Y07
WeakSeed       Y10
Name: 1938, dtype: object
Season        2015
Slot          R1Y8
StrongSeed     Y08
WeakSeed       Y09
Name: 1939, dtype: object
Season        2015
Slot          R1Z1
StrongSeed     Z01
WeakSeed       Z16
Name: 1940, dtype: object
Season        2015
Slot          R1Z2
StrongSeed     Z02
WeakSeed       Z15
Name: 1941, dtype: object
Season        2015
Slot          R1Z3
StrongSeed     Z03
WeakSeed       Z14
Name: 1942, dtype: object
Season        2015
Slot          R1Z4
StrongSeed     Z04

Name: 1929, dtype: object
Season        2015
Slot          R1X7
StrongSeed     X07
WeakSeed       X10
Name: 1930, dtype: object
Season        2015
Slot          R1X8
StrongSeed     X08
WeakSeed       X09
Name: 1931, dtype: object
Season        2015
Slot          R1Y1
StrongSeed     Y01
WeakSeed       Y16
Name: 1932, dtype: object
Season        2015
Slot          R1Y2
StrongSeed     Y02
WeakSeed       Y15
Name: 1933, dtype: object
Season        2015
Slot          R1Y3
StrongSeed     Y03
WeakSeed       Y14
Name: 1934, dtype: object
Season        2015
Slot          R1Y4
StrongSeed     Y04
WeakSeed       Y13
Name: 1935, dtype: object
Season        2015
Slot          R1Y5
StrongSeed     Y05
WeakSeed       Y12
Name: 1936, dtype: object
Season        2015
Slot          R1Y6
StrongSeed     Y06
WeakSeed       Y11
Name: 1937, dtype: object
Season        2015
Slot          R1Y7
StrongSeed     Y07
WeakSeed       Y10
Name: 1938, dtype: object
Season        2015
Slot          R1Y8
StrongSeed     Y08

In [11]:
year = min_year

simulator_scores = np.zeros(len(year_range))
simulator_accuracys = np.zeros(len(year_range))

# analyze results for all simulations
i = 0
for simulator in simulators:
    # get data from our db
    seeds_year = seeds_arr[year-1985] 
    slots_year = slots_arr[year-1985] 
    games_year = games_arr[year-1985]
    
    # get actual models
    actual_model = mmm.ActualTournament(games_arr[year-1985])
    actual_tourney = mmc.Tournament(seeds_year, slots_year, actual_model, include_scoring_dif=False)
    
    # score tournament 
    simulator_scores[i], simulator_accuracys[i] = simulator.score_tournament(actual_tourney, print_res=False)
    
    year = year + 1
    i = i + 1

Season        2014
Slot          R1W1
StrongSeed     W01
WeakSeed       W16
Name: 1849, dtype: object
Season        2014
Slot          R1W2
StrongSeed     W02
WeakSeed       W15
Name: 1850, dtype: object
Season        2014
Slot          R1W3
StrongSeed     W03
WeakSeed       W14
Name: 1851, dtype: object
Season        2014
Slot          R1W4
StrongSeed     W04
WeakSeed       W13
Name: 1852, dtype: object
Season        2014
Slot          R1W5
StrongSeed     W05
WeakSeed       W12
Name: 1853, dtype: object
Season        2014
Slot          R1W6
StrongSeed     W06
WeakSeed       W11
Name: 1854, dtype: object
Season        2014
Slot          R1W7
StrongSeed     W07
WeakSeed       W10
Name: 1855, dtype: object
Season        2014
Slot          R1W8
StrongSeed     W08
WeakSeed       W09
Name: 1856, dtype: object
Season        2014
Slot          R1X1
StrongSeed     X01
WeakSeed       X16
Name: 1857, dtype: object
Season        2014
Slot          R1X2
StrongSeed     X02
WeakSeed       X15
Name: 

In [13]:
print(simulator_scores)

[580. 990.]


In [18]:
year = min_year
greedy_tourneys   = []
top_seed_tourneys = []
actual_tourneys   = []

# analyze results for all simulations
i = 0
for year in year_range:
    # get data from our db
    seeds_year = seeds_arr[year-1985] 
    slots_year = slots_arr[year-1985] 
    games_year = games_arr[year-1985]
    
    # get actual models
    actual_model = mmm.ActualTournament(games_arr[year-1985])
    actual_tourneys.append(mmc.Tournament(seeds_year, slots_year, actual_model, include_scoring_dif=False))
    
    # get greedy models
    greedy_model = mmm.ModelPredictor(models[i], scalers[i], predictor_dfs, year, simulation=False)
    greedy_tourneys.append(mmc.Tournament(seeds_year, slots_year, greedy_model, include_scoring_dif=False))
    
    # get top seed models
    top_seed_model = mmm.BasicPredictor()
    top_seed_tourneys.append(mmc.Tournament(seeds_year, slots_year, top_seed_model, include_scoring_dif=False))
    
    year = year + 1
    i = i + 1

Season        2014
Slot          R1W1
StrongSeed     W01
WeakSeed       W16
Name: 1849, dtype: object
Season        2014
Slot          R1W2
StrongSeed     W02
WeakSeed       W15
Name: 1850, dtype: object
Season        2014
Slot          R1W3
StrongSeed     W03
WeakSeed       W14
Name: 1851, dtype: object
Season        2014
Slot          R1W4
StrongSeed     W04
WeakSeed       W13
Name: 1852, dtype: object
Season        2014
Slot          R1W5
StrongSeed     W05
WeakSeed       W12
Name: 1853, dtype: object
Season        2014
Slot          R1W6
StrongSeed     W06
WeakSeed       W11
Name: 1854, dtype: object
Season        2014
Slot          R1W7
StrongSeed     W07
WeakSeed       W10
Name: 1855, dtype: object
Season        2014
Slot          R1W8
StrongSeed     W08
WeakSeed       W09
Name: 1856, dtype: object
Season        2014
Slot          R1X1
StrongSeed     X01
WeakSeed       X16
Name: 1857, dtype: object
Season        2014
Slot          R1X2
StrongSeed     X02
WeakSeed       X15
Name: 

TypeError: __init__() missing 1 required positional argument: 'seeds_df'

In [17]:
greedy_tourneys

[]

In [16]:
# buffers
greedy_scores = np.zeros(len(year_range))
greedy_accuracys = np.zeros(len(year_range))

top_seed_scores = np.zeros(len(year_range))
top_seed_accuracys = np.zeros(len(year_range))

# analyze results 
year = min_year
i = 0
for i in range(len(year_range)):
    print(i)
    # get data from our db
    seeds_year = seeds_arr[year-1985] 
    slots_year = slots_arr[year-1985] 
    games_year = games_arr[year-1985]
    
    # get models
    actual_tourney = actual_tourneys[i]
    greedy_tourney = greedy_tourneys[i]
    top_seed_tourney = top_seed_tourneys[i]
    
    # score tournaments
    greedy_scores[i], greedy_accuracys[i] = greedy_tourney.score_tournament(actual_tourney, print_res=False)
    top_seed_scores[i], top_seed_accuracys[i] = top_seed_tourney.score_tournament(actual_tourney, print_res=False)
    
    year = year + 1
    i = i + 1

0


IndexError: list index out of range

In [34]:
print greedy_scores

[ 610.  980.]


In [51]:
celeb_data = pd.read_csv("datasets/our_data/celeb_brackets_2015.csv")
celeb_data

,person,points,percentile
0,Cousin Sal - ESPN,1200,92.1
1,J.A. Adande - ESPN,1090,87.0
2,MatthewBerry - ESPN,1000,77.7
3,Jeff Goodman - ESPN,990,76.5
4,Scott Van Pelt - ESPN,990,76.5
5,Vanessa Marshall - ESPN,970,73.9
6,Taylor Gray - ESPN,960,72.5
7,Mike Golic - ESPN,950,71.1
8,Dave Rothenberg - ESPN,920,66.6
9,Michael Smith - ESPN,910,65.1
